<a href="https://colab.research.google.com/github/sit-xinli/ai-course5/blob/main/LLM_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 大規模言語モデルのファインチューニング
LLMが唐詩を書けるように、あなたのLLMを微調整します。.

**TODOs**
1. スライドを読み、この宿題の目的を確認してください。
2. このColabノートをコピーして保存してください。
3. このColabノートの手順に従って、LLMを微調整する。


## GPUをアクティブにする

モデルを微調整するので、この宿題が妥当な時間（1～2時間）でできるように、GPUをアクティブにする必要があります。

T4 GPUを使っていることを確認！


## 事前データ準備
Tang_testingdata_ja.jsonとTang_trainingdata_ja.jsonをダウンロード

In [1]:
# @title
#if not os.path.exists('dataset/'):
!git clone https://github.com/sit-xinli/dataset.git
!ls /content

Cloning into 'dataset'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 3), reused 10 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 122.93 KiB | 736.00 KiB/s, done.
Resolving deltas: 100% (3/3), done.
dataset  sample_data


## パッケージのインストール
私たちは、微調整を容易にするために、他の人が作成したよくできたパッケージをインストールし、インポートします。

以下のコードブロックの実行にかかる時間は約 **5**分ですが、Colabの状態によって異なる場合があります。

In [2]:
""" このセルでコードを変更しないことを推奨します。 """
!pip install bitsandbytes
!pip install transformers
!pip install peft
!pip install sentencepiece
!pip install colorama
!pip install fsspec
!pip install huggingface_hub[hf_xet]
!pip install -U datasets
!pip install -U accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

以下のコードブロックの実行時間は約**20**秒ですが、Colabの状態によって異なる場合があります。

In [3]:
""" このセルでコードを変更しないことを推奨します。 """

import os
import sys
import argparse
import json
import warnings
import logging
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
#import bitsandbytes as bnb
from datasets import load_dataset, load_from_disk
import transformers, datasets
from peft import PeftModel
from colorama import *

from tqdm import tqdm
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import GenerationConfig
from peft import (
    #prepare_model_for_int8_training,
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training
)

## 微調整用データセットのダウンロード

## 便利な関数の定義

In [4]:
from google.colab import auth
auth.authenticate_user()

import google.auth
from googleapiclient.discovery import build

creds, _ = google.auth.default()
service = build('oauth2', 'v2', credentials=creds)
user_info = service.userinfo().get().execute()
email = user_info['email']
print("Authenticated as:", email)
import hashlib

# Hash the email and convert to a number
hash_digest = hashlib.sha256(email.encode()).hexdigest()
numeric_value = int(hash_digest, 16)

""" このセルでコードを変更しないことを推奨します。 """

seed = numeric_value % 101
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

Authenticated as: xinli@center.shonan-it.ac.jp


In [5]:

# トレーニングデータの作成
def generate_training_data(data_point):
    """
    (1) 目的
        - この関数は、データポイント（入力テキストと出力テキスト）を、モデルが読み取れるトークンに変換するために使用される。
    (2) 引数
        - data_point: dict。フィールドは "instruction"、"input"、"output"。
    (3) 返り値
        - モデルの入力トークン、モデルを因果的にするアテンションマスク、対応する出力ターゲットを持つdict
    (3) 例：
        - フィールド "instruction"、"input"、"output "がすべてstrであるdict、data_point_1を作成した場合、この関数は次のように使うことができる：
            formulate_article(データ_point_1)
    """
    # construct full input prompt
    prompt = f"""\
[INST] <<SYS>>
あなたは親切なアシスタントだし、唐詩を書くこともうまい。
<</SYS>>

{data_point["instruction"]}
{data_point["input"]}
[/INST]"""

    # count the number of input tokens
    len_user_prompt_tokens = (
        len(
            tokenizer(
                prompt,
                truncation=True,
                max_length=CUTOFF_LEN + 1,
                padding="max_length",
            )["input_ids"]
        ) - 1
    )
    # transform input prompt into tokens
    full_tokens = tokenizer(
        prompt + " " + data_point["output"] + "</s>",
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )["input_ids"][:-1]
    return {
        "input_ids": full_tokens,
        "labels": [-100] * len_user_prompt_tokens
        + full_tokens[len_user_prompt_tokens:],
        "attention_mask": [1] * (len(full_tokens)),
    }

# 生成された回答の評価
def evaluate(instruction, generation_config, max_len, input="", verbose=True):
    """
    (1) 目標
        - この関数は、与えられた入力文字列からモデルの出力を得るために使われる。

    (2) 引数：
        - instruction: str, モデルに何をさせたいかの説明。
        - generation_config: transformers.GenerationConfigオブジェクト、モデルの推論に関連するデコードパラメータを指定する。
        - max_len: int, モデルの出力の最大長。
        - input: str, モデルが命令を解くために必要な入力文字列、デフォルトは""(入力なし)
        - verbose: bool, モードの出力を表示するかどうか、デフォルトはTrue
    (3) 戻り値
        - output: str, 命令と入力に従ったモードの応答
    (4) 例
        - 命令が "ABC"、入力が "DEF "で、128トークン以下の回答をモデルに与えたい場合、この関数を次のように使うことができる：
            evaluate(instruction="ABC", generation_config=generation_config, max_len=128, input="DEF")

    """
    # construct full input prompt
    prompt = f"""\
[INST] <<SYS>>
あなたは親切なアシスタントだし、唐詩を書くこともうまい。
<</SYS>>

{instruction}
{input}
[/INST]"""
    # プロンプトのテキストをモデルが必要とする数値表現に変換する。
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    # モデルを使って返信を生成する
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_len,
    )
    # 生成された応答をデコードしてプリントアウトする。
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        output = output.split("[/INST]")[1].replace("</s>", "").replace("<s>", "").replace("Assistant:", "").replace("Assistant", "").strip()
        if (verbose):
            print(output)

    return output


## 微調整前のモデルと推論をダウンロードする

以下のコードブロックの実行時間は、デフォルトの設定を使用した場合、約 **10**分かかりますが、Colabの状態によって異なる場合があります。

## 微調整前の推論
まず、ファインチューニングなしのモデルで何ができるかを見てみよう。

以下のコードブロックの実行時間は、デフォルトの設定を使用した場合、約2分**かかりますが、Colabの状態によって異なる場合があります。

In [6]:
""" このセルでコードを変更しないことを推奨します。 """

model_name = "Qwen/Qwen3-0.6B" # 微調整に使用するモデルを設定する。
cache_dir = "./cache"

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# 指定されたモデル名またはパスから，事前に学習された言語モデルを読み込みます．
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir=cache_dir,
    quantization_config=nf4_config,
    low_cpu_mem_usage = True,
    use_auth_token=False
)

# トークナイザーを作成し、終了シンボル(eos_token)を設定します。
logging.getLogger('transformers').setLevel(logging.ERROR)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    add_eos_token=True,
    cache_dir=cache_dir,
    #quantization_config=nf4_config
    use_auth_token=False
)
tokenizer.pad_token = tokenizer.eos_token

# モデル推論のためのデコーディング・パラメータの設定
max_len = 128
generation_config = GenerationConfig(
    do_sample=True,
    temperature=0.1,
    num_beams=1,
    top_p=0.3,
    no_repeat_ngram_size=3,
    pad_token_id=2,
)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

以下のコードブロックは、デフォルトの設定を使用した場合、実行に約 **1** 分かかりますが、Colab の状態によって異なる場合があります。

In [7]:
""" このセルでコードを変更しないことを推奨します。 """

# demo examples
test_tang_list = ['会っても別れを告げるのは難しい，東風には力がなく、花はすべて散ってしまった。',
                  '重いカーテンの下で、深い喪に服していた，横になってからの夜は長く、澄んでいる。',
                  '逃亡の果てに追いかける香りの星，禁断の園は驚きに満ちている。']

system_prompt = '以下は唐詩の一行目である。 あなたの知識で判断し、唐詩を簡潔に完成しなさい。'

# get the model output for each examples
demo_before_finetune = []
for tang in test_tang_list:
  demo_before_finetune.append(f'モデル入力:\n{system_prompt}\n\n{tang}\n\nモデル出力:\n'+evaluate(system_prompt, generation_config, max_len, tang, verbose = False))

# print and store the output to text file
for idx in range(len(demo_before_finetune)):
  print(f"Example {idx + 1}:")
  print(demo_before_finetune[idx])
  print("-" * 80)


Example 1:
モデル入力:
以下は唐詩の一行目である。 あなたの知識で判断し、唐詩を簡潔に完成しなさい。

会っても別れを告げるのは難しい，東風には力がなく、花はすべて散ってしまった。

モデル出力:
この詩は、詩の内容をより簡潔にするために、詩を短くした。  
詩の文脈を把握し、詩に含まれている名前や人物や物語を簡素に記述し、  
詩に必要な情報を簡潔にして、  
この唐詩は「東風の力が強いため、東風が花を散った」と簡潔な表現にまとめられます。

この詩の名前、人物、物語、テーマ、内容、文脈、テーマの内容、テーマをまとめると、  
**「東风の力」**  
**
--------------------------------------------------------------------------------
Example 2:
モデル入力:
以下は唐詩の一行目である。 あなたの知識で判断し、唐詩を簡潔に完成しなさい。

重いカーテンの下で、深い喪に服していた，横になってからの夜は長く、澄んでいる。

モデル出力:
この詩は、詩の名前は「重いのカーヤントの下に深い喪に着ていた」という詩です。

この詩の内容を簡約にまとめると、  
**「重くしたカーボンの上に深いた喪にいた、夜は長い」**  
となります。

この唐詩は「長く夜の深さ」を表しています。

この诗の内容は、  
「重いたカーリンの夜は深い」  
と簡潔でまとめられています。

詩の文脈を考慮して、  
この唐诗の名
--------------------------------------------------------------------------------
Example 3:
モデル入力:
以下は唐詩の一行目である。 あなたの知識で判断し、唐詩を簡潔に完成しなさい。

逃亡の果てに追いかける香りの星，禁断の園は驚きに満ちている。

モデル出力:
この詩は、詩の名前と内容を記述して、簡潔で、一言でまとめ、一観をまとめ、まとめ、整理、整理して、整理し、整理した内容をまとめると、  
**「**  
**詩名：**  
「**逃亡**の果たに追かえる香りにのる星，**禁断**の園に驚き**に溁ちている。**」  
**内容：** 這詩は，詩の内容を簡素にまとめると

## 微調整のためのハイパーパラメーターの設定



In [8]:
""" このハイパーパラメータで遊んでみることを強くお勧めする。 """

num_train_data = 1040 # ほとんどの場合, 可能な限り多くのデータを訓練したいでしょう. これにより, モデルがより多様な節を見ることができるようになり, 出力の質が向上しますが, 訓練時間も長くなります.
                      # デフォルトのパラメータ(1040)を使用した場合: 微調整に約25分、全セルのフル稼働に約50分かかる。
                      # 最大値(5000)を使用した場合: 微調整には約100分かかり, 全セルのフル実行には約120分かかる.

""" これらのハイパーパラメータのいくつかを変更したいかもしれない（必ずしも必要ではない）。 """

output_dir = "/content/dataset"  # 結果を出力するディレクトリを設定する（別のディレクトリに結果を保存したい場合は、ここで変更できますが、デフォルトのサブディレクトリ、つまりGoogleドライブに保存することを強くお勧めします）
ckpt_dir = "./exp1" # モデルのチェックポイントを保存するディレクトリを設定します（モデルのチェックポイントを別のディレクトリに保存したい場合は、ここで変更できます）。
num_epoch = 1 # 学習する総エポック数を設定する（数値が大きいほど学習時間が長くなる。colabの無料版を利用する場合、学習時間が長すぎると切断される可能性があるので注意が必要）。
LEARNING_RATE = 3e-4 # 学習率を設定する。


""" このパラメータ設定コードを変更しないことを推奨します。 """

cache_dir = "./cache" # キャッシュディレクトリのパスを設定します。
from_ckpt = False # チェックポイントからモデルの重みをロードするかどうか, デフォルトはno.
ckpt_name = None # 特定のチェックポイントから重みをロードする際に使用するファイル名、デフォルトはなし。
dataset_dir = "/content/dataset/Tang_trainingdata_ja.json" # データセットのディレクトリまたはファイルパスを設定します．
logging_steps = 20 # 学習ログを出力するステップ数を定義します。
save_steps = 65 # モデルを保存するステップ数を設定します。
save_total_limit = 3 # モデルのチェックポイントを最大何回保持するかを制御します。
report_to = None # 実験的メトリクスを報告する対象を設定します。
MICRO_BATCH_SIZE = 4 # マイクロバッチのサイズを定義する
BATCH_SIZE = 16 # バッチのサイズを定義する
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE # 各マイクロバッチの累積グラデーションステップ数を計算する
CUTOFF_LEN = 256 # テキストカットオフの最大長を設定します.
LORA_R = 8 # LORA(Layer-wise Random Attention)のR値を設定します.
LORA_ALPHA = 16 # LORAのアルファ値を設定します.
LORA_DROPOUT = 0.05 # LORAのドロップアウト率を設定する。
VAL_SET_SIZE = 0 # バリデーションセットのサイズを設定します。
TARGET_MODULES = ["q_proj", "up_proj", "o_proj", "k_proj", "down_proj", "gate_proj", "v_proj"] # ターゲットとなるモジュールを設定する。
device_map = "auto" # デバイスマップを設定。デフォルトは "auto"。
world_size = int(os.environ.get("WORLD_SIZE", 1)) # 環境変数 "WORLD_SIZE "の値を取得、設定されていない場合はデフォルトで1。
ddp = world_size != 1 # world_sizeに基づいて分散データ処理(DDP)を使用するかどうかを判断。world_sizeが1の場合、DDPは使用されない。
if ddp:
    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
    GRADIENT_ACCUMULATION_STEPS = GRADIENT_ACCUMULATION_STEPS // world_size

## 微調整開始
以下のコードブロックの実行時間は、デフォルト設定を使用した場合、約**10分**かかりますが、Colabの状態によって異なる場合があります。
微調整の方法は、以下のサイトを参考：https://www.datacamp.com/tutorial/fine-tuning-qwen3

In [9]:
""" このセルでコードを変更しないことを推奨します。 """
os.environ["WANDB_MODE"] = "offline" # disables online logging
os.environ["WANDB_DISABLED"] = "true" # disables wandb entirely

# create the output directory you specify
os.makedirs(output_dir, exist_ok = True)
os.makedirs(ckpt_dir, exist_ok = True)

# 根據 from_ckpt 標誌，從 checkpoint 載入模型權重
if from_ckpt:
    model = PeftModel.from_pretrained(model, ckpt_name)

# 將模型準備好以使用 INT8 訓練
model = prepare_model_for_kbit_training(model)

# 使用 LoraConfig 配置 LORA 模型
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

# トークナイザー 的 パディング トークン を 0に設定する
tokenizer.pad_token_id = 0

# トレーニングデータのロードと処理
with open(dataset_dir, "r", encoding = "utf-8") as f:
    data_json = json.load(f)
with open("tmp_dataset.json", "w", encoding = "utf-8") as f:
    json.dump(data_json[:num_train_data], f, indent = 2, ensure_ascii = False)

data = load_dataset('json', data_files="tmp_dataset.json", download_mode="force_redownload")

# 学習データを学習セットと検証セットに分割する（VAL_SET_SIZEが0より大きい場合）
if VAL_SET_SIZE > 0:
    train_val = data["train"].train_test_split(
        test_size=VAL_SET_SIZE, shuffle=True, seed=42
    )
    train_data = train_val["train"].shuffle().map(generate_training_data)
    val_data = train_val["test"].shuffle().map(generate_training_data)
else:
    train_data = data['train'].shuffle().map(generate_training_data)
    val_data = None

# トランスフォーマー・トレーナーによるモデル・トレーニング
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=50,
        num_train_epochs=num_epoch,
        learning_rate=LEARNING_RATE,
        fp16=True,  # 混合精度トレーニングの使用
        logging_steps=logging_steps,
        save_strategy="steps",
        save_steps=save_steps,
        output_dir=ckpt_dir,
        save_total_limit=save_total_limit,
        ddp_find_unused_parameters=False if ddp else None,  # DDPを使用して勾配更新戦略を制御するかどうか
        report_to=report_to,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# モデルのキャッシュ機能を無効にする
model.config.use_cache = False

# PyTorchバージョン2.0以降とWindows以外のシステムを使用している場合のモデルのコンパイル
if torch.__version__ >= "2" and sys.platform != 'win32':
    model = torch.compile(model)

# モデルトレーニングの開始
trainer.train()

# 学習済みモデルを指定したディレクトリに保存する。
model.save_pretrained(ckpt_dir)

# トレーニング中にウェイトが不足する可能性があるという警告メッセージを表示する。
print("\n 上記のキーが見つからないという警告は無視してください :)")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

{'loss': 4.0941, 'grad_norm': 2.426934003829956, 'learning_rate': 0.00011399999999999999, 'epoch': 0.3076923076923077}
{'loss': 2.5443, 'grad_norm': 1.0314195156097412, 'learning_rate': 0.000234, 'epoch': 0.6153846153846154}
{'loss': 2.2902, 'grad_norm': 0.9585222601890564, 'learning_rate': 0.00011999999999999999, 'epoch': 0.9230769230769231}


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

{'train_runtime': 207.0171, 'train_samples_per_second': 5.024, 'train_steps_per_second': 0.314, 'train_loss': 2.9242810029249924, 'epoch': 1.0}

 上記のキーが見つからないという警告は無視してください :)


##  テスト
微調整は終わった。調整後のモデルをテストしたい。

まず、保存した微調整済みモデル（チェックポイント）をロードする必要がある。

In [10]:
""" このセルでコードを変更しないことを推奨します。 """

# find all available checkpoints
ckpts = []
for ckpt in os.listdir(ckpt_dir):
    if (ckpt.startswith("checkpoint-")):
        ckpts.append(ckpt)

# list all the checkpoints
ckpts = sorted(ckpts, key = lambda ckpt: int(ckpt.split("-")[-1]))
print("all available checkpoints:")
print(" id: checkpoint name")
for (i, ckpt) in enumerate(ckpts):
    print(f"{i:>3}: {ckpt}")


all available checkpoints:
 id: checkpoint name
  0: checkpoint-65


In [11]:
""" チェックポイントを変更したいと思うかもしれないが、必ずしも必要ではない。"""

id_of_ckpt_to_use = -1 # 推論に使用するチェックポイントのID（前のセルの出力に対応）。
                        # デフォルト値の-1は, 上記のチェックポイントのリストの中で "最後から2番目 "のチェックポイントを指します.
                        # 他のチェックポイントを選択したい場合は, -1をリストにあるチェックポイントIDのどれかに変更します.

ckpt_name = os.path.join(ckpt_dir, ckpts[id_of_ckpt_to_use])

""" デコード・パラメータを変更する必要があるかもしれないが、必ずしも必要ではない。 """

# ここでデコードパラメータを調整することができます。デコードパラメータの詳細な説明については、宿題のスライドを参照してください。
max_len = 128 # 生成される返信の最大長。
temperature = 0.1 # 生成される返信のランダム性を設定。値が小さいほど返信が安定する。
top_p = 0.3 # top-p(核)サンプリングのしきい値.
# top_k = 5 # top-kの値を調整することで、生成される返答の多様性を高め、繰り返し単語が 生成されないようにする。

以下のコードブロックの実行時間は、デフォルト設定を使用した場合、約2分**かかりますが、Colabの状態によって異なる場合があります。

In [12]:
""" このセルでコードを変更しないことを推奨します。 """

test_data_path = "/content/dataset/Tang_testingdata_ja.json"
output_path = os.path.join(output_dir, "results.txt")

cache_dir = "./cache" # キャッシュディレクトリのパスを設定する.
seed = 42 # 結果を再現するためのランダムシードを設定する。
no_repeat_ngram_size = 3 # 重複セグメントを生成しないように、no-repeat ngramのサイズを設定する。

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# トークン化器を使用して、モデル名をモデルが読み取り可能な数値表現に変換します。
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=cache_dir,
    quantization_config=nf4_config
)

# 事前学習モデルからモデルをロードし、8ビット整数(INT8)モデルとして設定する。
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=nf4_config,
    device_map={'': 0},  # 設定使用的設備，此處指定為 GPU 0
    cache_dir=cache_dir
)

# 指定したチェックポイントからモデルの重みをロードする
model = PeftModel.from_pretrained(model, ckpt_name, device_map={'': 0})

以下のコードブロックの実行時間は、デフォルト設定を使用した場合、約 **4**分かかりますが、Colabの状態によって異なる場合があります。

In [13]:
""" このセルでコードを変更しないことを推奨します。 """

results = []

# ランダム性、ビームサーチ、その他の関連パラメータを含む生成コンフィギュレーションを設定する。
generation_config = GenerationConfig(
    do_sample=True,
    temperature=temperature,
    num_beams=1,
    top_p=top_p,
    # top_k=top_k,
    no_repeat_ngram_size=no_repeat_ngram_size,
    pad_token_id=2
)

# テストデータの読み込み
with open(test_data_path, "r", encoding = "utf-8") as f:
    test_datas = json.load(f)

# 各テストデータに対して予測を行い、結果を保存する。
with open(output_path, "w", encoding = "utf-8") as f:
  for (i, test_data) in enumerate(test_datas):
      predict = evaluate(test_data["instruction"], generation_config, max_len, test_data["input"], verbose = False)
      f.write(f"{i+1}. {test_data['input']} ★★★ {predict}\n")
      print(f"{i+1}. {test_data['input']} ★★★ {predict}")


1. 雪が晴れて銀の衣装を纏い、素朴な景色が広がる。橘の木は高くそびえ、瓊の枝に映える。 ★★★ 金の衣を揺らし、玉の衣の裾を揃かせる。玉の枝は青の花を映し、金の枝の花は白い花を揠る。遠くの山は青く、遠くに山は白く。遠いの道は青い、遠い道は白か。 金と玉の花が揺れる、青と白の枝が映える。遠かの山の下に、青く花が咲き、白く花は揺れている。. 金は
2. 夫子は何を為す者ぞ？一代の中にひたすらに暮らす。 ★★★ 一朝の日を望み、二月の日はまた望みる。ただ春の花を賞すと、秋の花はまた賞す。 一時の花の色は、二時に花の香りが広がる。春の香はまだ未だ、秋は香は広く広かに。. 今や花の花が開き、今や香の香が庁を満たす。</strong> 今朝の花、今朝に開く。 花の
3. 飛び蓋は芳園を去り、蘭は翠渚を遊ぶ。 ★★★ 一朝の花は芳苑の外に咲き、二月の花が翠渚に咵り咲く。花の色は青い、花の形は紫の。花は香りを帯び、香りは香の香りに満ちる。 一時の花の香は芳の園に咼き、花は翠の渚に遊ぶ。花が香りの香を帓し、香は香を満たす。芳の花を咲かせ、翠の
4. 風が節句を告げて吹き始め、灰色の律動が初陽とともに動き出す。 ★★★ 一時の春の花は、まだ開き方を待つ。花の色はまだ青いまま、花の香はまだ香りを残す。風はまだ吹き去り、花はまだ開いていない。 一朝の花の開き、二日は花の閉じ。風の声はまだ、花を吹き散らす。
花はまた開き始めて、風はまた吹き始める。花は開き閉じ、風の聲は吹き、花が閉じて閉じ
5. 昨夜の星辰、昨夜の風  
画楼の西、桂堂の東 ★★★ 月はまだ夜明けの前、星はまだ明るく夜明るい。画楼西に、桂の堂東に。夜明く、月は夜明かり、星が夜明かし。 月の色はまだ未だ明るさを帯び、星の色もまだ夜の明かりを席し、画楼は西にあり、桂は東にあり。月の光はまだまだ夜を帓し、星を帊し夜明き。 星は夜の
6. 三日厨に入り、手を洗い、羹と湯を作る。 ★★★ 一朝の日、三日を祝う。日を洗うと、湯を煮す。日が明るく、湯は清潔に。 一夕の日を、三つの日を賜う。湯を洗くと、日を煮る。日が清潔で、湯が清かし。 花の香りは、香りを散らす。
花の香は香り散らし、香は散らばる。</p> また、香の香が散ら
7. 嵩山の雲、秦の樹、長く離れ住む  
双鯉、遠く遠く、一通の手紙 ★★★ 一縦の水を流し、一縋の木を植す  

## **重要なこと**： 15個の唐詩の結果を提出 .
これらの唐詩の結果は "/content/dataset/results.txt "にあります。

## ファインチューニング・モデルとファインチューニングなしのモデルの比較をご覧ください。

ここで、上の「微調整前の推論」で見たのと同じ例で、我々のモデルがどのようなことができるかをチェックする。

以下のコードブロックの実行時間は、デフォルトの設定であれば**40**秒程度ですが、Colabの状態によって異なるかもしれません。ここで、上の「微調整前の推論」で見たのと同じ例で、我々のモデルがどのようなことができるかをチェックする。



In [14]:
# test demo examples
test_tang_list = ['会っても別れを告げるのは難しい，東風には力がなく、花はすべて散ってしまった。',
                  '重いカーテンの下で、深い喪に服していた，横になってからの夜は長く、澄んでいる。',
                  '逃亡の果てに追いかける香りの星，禁断の園は驚きに満ちている。']

# get the model output for each examples
demo_before_finetune = []
for tang in test_tang_list:
  demo_before_finetune.append(f'モデル入力:\n{system_prompt}\n\n{tang}\n\nモデル出力:\n'+evaluate(system_prompt, generation_config, max_len, tang, verbose = False))

# print and store the output to text file
for idx in range(len(demo_before_finetune)):
  print(f"Example {idx + 1}:")
  print(demo_before_finetune[idx])
  print("-" * 80)


Example 1:
モデル入力:
以下は唐詩の一行目である。 あなたの知識で判断し、唐詩を簡潔に完成しなさい。

会っても別れを告げるのは難しい，東風には力がなく、花はすべて散ってしまった。

モデル出力:
一朝の夕暮れ、二月の花は散り、三月の風は散らばる。春の花が散らされ、秋の花の香は散る。ただ、花の色は散っただけ。 一夕の夕、二つの花は広がる。三つの風は庁を散らす。春は散れ、秋は香を散る。. 一時の夕陽、二度の花を広げ、三度の風を散り。春を散らせ、秋を香に
--------------------------------------------------------------------------------
Example 2:
モデル入力:
以下は唐詩の一行目である。 あなたの知識で判断し、唐詩を簡潔に完成しなさい。

重いカーテンの下で、深い喪に服していた，横になってからの夜は長く、澄んでいる。

モデル出力:
一時の哀しみを抱き、長く夜を待つ。ただ一人の哀れことを抱きつつ、夜は澄み、月は明るい。 一縦の哀しを抱く、一人の夜を望む。ただ哀しきことを抱いて、夜の月は澄か。. 一寸の哀しさを抱いていた、一人を望み、夜を待ち。 楚の月の夜、一人に哀しだい、月の澄みに哀し。
--------------------------------------------------------------------------------
Example 3:
モデル入力:
以下は唐詩の一行目である。 あなたの知識で判断し、唐詩を簡潔に完成しなさい。

逃亡の果てに追いかける香りの星，禁断の園は驚きに満ちている。

モデル出力:
一時の春の花は、まだ花の色を残す。また、春の夜は、花の香りを満たす。ただ、花はまだ開き、香はまだ香り。また花は開き続け、香りは香り続け。 一朝の花の花が開き始めて、香の香は香われて。また香の花を咲かせる、香を咵らせる。. また花の開き方を知らず、香が香われている。. 花
--------------------------------------------------------------------------------


## **重要**： 上記の3つの例は提出しないでください。
この3つの例は、微調整前と微調整後のモデルのパフォーマンスを比較するためだけに使用されます。


## 結果のあなたのPCにダウンロードします（ダウンロードフォルダにresult.txtファイル）
宿題を終わらせるには、このファイルが必要です。 ブラウザが自動的にダウンロードしない場合は、Google Driveにあります。

In [15]:
from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>